# Execução do modelo

In [ ]:
from scripts.authentcation import authenticate_huggingface

authenticate_huggingface()

In [ ]:
from datasets import load_dataset

dataset = load_dataset('marmal88/skin_cancer', split='test')

In [ ]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    'unsloth/Llama-3.2-11B-Vision-Instruct',
    load_in_4bit=True,
    use_gradient_checkpointing='unsloth',
)

In [ ]:
from transformers import TextStreamer
from scripts.messages import add_inference_message

FastVisionModel.for_inference(model)

image = dataset[0]['image']

messages = add_inference_message('Classify the skin lesion in the image.')

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors='pt',
).to('cuda')

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128,
                   use_cache=True, temperature=1.0, min_p=0.1)